In [63]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import beta
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

   __ 1. На прошедшей неделе в рекламной сети параллельно размещалось два баннера. Оба баннера были показаны один миллион раз. Первый получил 10 000 кликов и 500 установок, а второй — 10 500 кликов и 440 установок. Маркетолог просит у вас совета: какой баннер оставить, а какой отключить? Что вы ему ответите? Обоснуйте свой ответ.__
   

In [21]:
class banner: 
    def __init__(self, views, click, install):
        self.click = click*1.
        self.install = install*1.
        self.views = views*1.
        self.p_click = click*1./views
        self.p_install = install*1./views     

In [26]:
banner_a = banner(10**6,10000,500)
banner_b = banner(10**6,10500,440)

#### Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [27]:
conf_interval_banner_a = proportion_confint(banner_a.click, 
                                            banner_a.views,
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(banner_b.click, 
                                            banner_b.views,
                                            method = 'wilson')

In [28]:
print '95%% confidence interval for a click probability, banner a: [%f, %f]' % conf_interval_banner_a
print '95%% confidence interval for a click probability, banner b [%f, %f]' % conf_interval_banner_b

95% confidence interval for a click probability, banner a: [0.009807, 0.010197]
95% confidence interval for a click probability, banner b [0.010302, 0.010702]


Доверительные интервалы не пересекаются. Можно с уверенностью 95% утверждать, что banner_b просматривают чаще. 

In [30]:
conf_interval_banner_a = proportion_confint(banner_a.install, 
                                            banner_a.views,
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(banner_b.install, 
                                            banner_b.views,
                                            method = 'wilson')

In [31]:
print '95%% confidence interval for an install probability, banner a: [%f, %f]' % conf_interval_banner_a
print '95%% confidence interval for an install probability, banner b [%f, %f]' % conf_interval_banner_b

95% confidence interval for an install probability, banner a: [0.000458, 0.000546]
95% confidence interval for an install probability, banner b [0.000401, 0.000483]


Доверительные интервалы пересекаются, проверим гипотезу разности долей для независимых выборок.

#### Z-критерий для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n$| $n$
  
$$ \hat{p}_1 = \frac{a}{n}$$

$$ \hat{p}_2 = \frac{b}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n} + \frac{1}{n})}}$$
$$P = \frac{\hat{p}_1 + \hat{p}_2}{2} $$

In [44]:
def proportions_diff_confint_ind(p1, p2, n, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
       
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n + p2 * (1 - p2)/ n)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n + p2 * (1 - p2)/ n)
    
    return (left_boundary, right_boundary)

In [48]:
def proportions_diff_z_stat_ind(p1, p2, n):
    P = float(p1 + p2) / 2
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (2./n))

In [34]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [56]:
print "95%% confidence interval for a difference between frequencies of installations: [%f, %f]" %\
      proportions_diff_confint_ind(banner_a.p_install, banner_b.p_install, banner_a.views)

95% confidence interval for a difference between frequencies of installations: [-0.000000, 0.000120]


In [51]:
print "p-value: %f" % proportions_diff_z_test( \
                      proportions_diff_z_stat_ind(banner_a.p_install, banner_b.p_install, banner_a.views))

p-value: 0.050295


Видимо данные приведены на момент остановки эксперимента и выбора баннера a. Округлив p-value до третьего знака можно сказать, 
что гипотеза о равенсве частот установок программы после просмотра баннера a и баннера b отвергается
на уровне значимости 0.05.  

Естественно, тот факт, что на баннер b статичтически значимо кликают чаще не должен нас останавливать.
Ведь наша конечная цель - установка программы, расположенной по ссылке.
К тому же меншьшее колличество кликов по баннеру a, видимо, говорит о том, что он лучше попадает в целевую
аудиторию. Народу переходит по ссылке меньше, а установок болше и народное время не сжигается впустую на изучение 
ненужных предложений.

Проверим ту же гипотезу с односторонней альтернативой, используя априорное предположение, что banner_a - лучше.

In [52]:
print "p-value: %f" % proportions_diff_z_test( \
                proportions_diff_z_stat_ind(banner_a.p_install, banner_b.p_install, banner_a.views), 'greater')

p-value: 0.025148


   __2. Есть гипотеза, что если прислать пользователю СМС-сообщение с напоминанием о сервисе, то он с некоторой вероятностью сделает заказ и станет в дальнейшем ездить чаще. Как проверить эту гипотезу и принять решение о том, следует ли использовать такой способ коммуникации?__

___Рассмотрим данную задачу как двуxвыборочную со связанными выборками. ___

Выделим случайным образом группу из общей аудитории сервиса, которой мы будет присылать смс-напоминание.
Найем доверительный интервал для разности средних недельных (месячных) доходов от пользователей за период до получения смс и после получения смс. 
Если допустить, что "недельный (месячный) доход от пользователя" нормально распределеннная случайная величина, то для проверки равенства математических ожиданий двух выборок можно использовать t-критерий Стьюдента.
Если не нормально, то для проверки равенства медиан воспользуемся критерием знаковых рангов Уилкоксона.

Необходимо заметить, что параметры распределения сучайной величины "недельный (месячный) доход от пользователя" могут зависеть от времени. 

___Рассмотреть задачу как двуxвыборочную с независимыми выборками___

В этом случае второй группой будут все пользователи не получавшие смс.
Так же построим доверительный интервал для разности средних недельных (месячных) доходов от пользователей в группах sms и no_sms. 
 
Колличество людей в группе sms определяется величиной эффекта который мы хотим измерить. 
 
Если допустить, что "недельные (месячные) доходы от пользователя" для групп sms и no_sms  -  нормально распределенные случайные величины дисперсия которых одинакова, а математическое ожидание отличается, то для расчета необходимого объема тестовой группы можно воспользоваться формулой: $$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

Если среднеквадратичное отклонение этой величины 500 р., то 3000 человек будет достаточно для получения доверительного интервала шириной +-18 р.

Не делая никаких предположений о распределении можно выполнить интервальную оценку разности средних с помощью bootstrap выборок. Для проверки гипотезы о разности средних значений использовать ранговый критерий Манна-Уитни.